In [4]:
!pip install tensorlayer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.2/381.2 kB 7.1 MB/s eta 0:00:00


In [11]:
import random
import time
import datetime
import numpy as np
import tensorflow as tf
import tensorlayer as tl
from tensorlayer.layers import Dense, Dropout, Flatten, Input
from tensorlayer.models import Model

num_classes = 10
epochs = 20
batch_size = 128
input_shape = (None, 784)

def contrastive_loss(label, feature1, feature2):
    margin = 1.0
    eucd = tf.sqrt(tf.reduce_sum(tf.square(feature1 - feature2), axis=1))
    return tf.reduce_mean(label * tf.square(eucd) + (1 - label) * tf.square(tf.maximum(margin - eucd, 0)))

def compute_accuracy(label, feature1, feature2):
    eucd = tf.sqrt(tf.reduce_sum((feature1 - feature2)**2, axis=1))
    pred = tf.cast(eucd < 0.5, label.dtype)
    return tf.reduce_mean(tf.cast(tf.equal(pred, label), tf.float32))

def create_base_network(input_shape):
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(128, act=tf.nn.relu)(x)
    x = Dropout(0.9)(x)
    x = Dense(128, act=tf.nn.relu)(x)
    x = Dropout(0.9)(x)
    x = Dense(128, act=tf.nn.relu)(x)
    return Model(input, x)

def get_siamese_network(input_shape):
    base_layer = create_base_network(input_shape).as_layer()
    ni_1 = Input(input_shape)
    ni_2 = Input(input_shape)
    nn_1 = base_layer(ni_1)
    nn_2 = base_layer(ni_2)
    return Model(inputs=[ni_1, ni_2], outputs=[nn_1, nn_2])

def create_pairs(x, digit_indices):
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels).astype(np.float32)

try:
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
    X_train = X_train.reshape(-1, 784).astype(np.float32) / 255.0
    X_test = X_test.reshape(-1, 784).astype(np.float32) / 255.0
    y_train = y_train.astype(np.int64)
    y_test = y_test.astype(np.int64)

    X_val = X_train[50000:]
    y_val = y_train[50000:]
    X_train = X_train[:50000]
    y_train = y_train[:50000]
    tl.logging.info("Data loaded and preprocessed successfully using tf.keras.")
except Exception as e:
    tl.logging.error(f"Failed to load data using tf.keras: {e}")
    exit()

digit_indices = [np.where(y_train == i)[0] for i in range(num_classes)]
tr_pairs, tr_y = create_pairs(X_train, digit_indices)
digit_indices = [np.where(y_val == i)[0] for i in range(num_classes)]
val_pairs, val_y = create_pairs(X_val, digit_indices)
digit_indices = [np.where(y_test == i)[0] for i in range(num_classes)]
te_pairs, te_y = create_pairs(X_test, digit_indices)

model = get_siamese_network(input_shape)
train_weights = model.trainable_weights
optimizer = tf.optimizers.RMSprop()

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/siamese/' + current_time + '/train'
val_log_dir = 'logs/siamese/' + current_time + '/validation'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
val_summary_writer = tf.summary.create_file_writer(val_log_dir)
tl.logging.info(f"TensorBoard logs will be saved to: logs/siamese/{current_time}")

print_freq = 5

@tf.function
def train_step(X_batch, y_batch):
    with tf.GradientTape() as tape:
        _out1, _out2 = model([X_batch[:, 0, :], X_batch[:, 1, :]])
        _loss = contrastive_loss(y_batch, _out1, _out2)
    grad = tape.gradient(_loss, train_weights)
    optimizer.apply_gradients(zip(grad, train_weights))
    _acc = compute_accuracy(y_batch, _out1, _out2)
    return _loss, _acc

@tf.function
def eval_step(X_batch, y_batch):
    _out1, _out2 = model([X_batch[:, 0, :], X_batch[:, 1, :]])
    _loss = contrastive_loss(y_batch, _out1, _out2)
    _acc = compute_accuracy(y_batch, _out1, _out2)
    return _loss, _acc

for epoch in range(epochs):
    start_time = time.time()

    train_loss, train_acc, n_iter = 0, 0, 0
    model.train()
    for X_batch, y_batch in tl.iterate.minibatches(tr_pairs, tr_y, batch_size, shuffle=True):
        _loss, _acc = train_step(X_batch, y_batch)
        train_loss += _loss
        train_acc += _acc
        n_iter += 1
    avg_train_loss = train_loss / n_iter
    avg_train_acc = train_acc / n_iter

    with train_summary_writer.as_default():
        tf.summary.scalar('loss', avg_train_loss, step=epoch)
        tf.summary.scalar('accuracy', avg_train_acc, step=epoch)

    val_loss, val_acc, n_iter_val = 0, 0, 0
    model.eval()
    for X_batch, y_batch in tl.iterate.minibatches(val_pairs, val_y, batch_size, shuffle=False):
        _loss, _acc = eval_step(X_batch, y_batch)
        val_loss += _loss
        val_acc += _acc
        n_iter_val += 1
    avg_val_loss = val_loss / n_iter_val
    avg_val_acc = val_acc / n_iter_val

    with val_summary_writer.as_default():
        tf.summary.scalar('loss', avg_val_loss, step=epoch)
        tf.summary.scalar('accuracy', avg_val_acc, step=epoch)

    if epoch + 1 == 1 or (epoch + 1) % print_freq == 0:
        print(f"Epoch {epoch + 1}/{epochs} took {time.time() - start_time:.2f}s")
        print(f"  Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f}")
        print(f"  Val Loss:   {avg_val_loss:.4f}, Val Acc:   {avg_val_acc:.4f}")

# --- بخش ۶: ارزیابی نهایی روی داده‌های تست ---
print("\nStarting final test evaluation...")
model.eval()
test_loss, test_acc, n_iter_test = 0, 0, 0
for X_batch, y_batch in tl.iterate.minibatches(te_pairs, te_y, batch_size, shuffle=False):
    _loss, _acc = eval_step(X_batch, y_batch)
    test_loss += _loss
    test_acc += _acc
    n_iter_test += 1
print(f"  Test Loss: {test_loss / n_iter_test:.4f}")
print(f"  Test Acc:  {test_acc / n_iter_test:.4f}")

Epoch 1/20 took 11.35s
  Train Loss: 0.0820, Train Acc: 0.9070
  Val Loss:   0.0381, Val Acc:   0.9575
Epoch 5/20 took 7.41s
  Train Loss: 0.0171, Train Acc: 0.9828
  Val Loss:   0.0233, Val Acc:   0.9728
Epoch 10/20 took 7.57s
  Train Loss: 0.0102, Train Acc: 0.9895
  Val Loss:   0.0233, Val Acc:   0.9732
Epoch 15/20 took 6.12s
  Train Loss: 0.0078, Train Acc: 0.9922
  Val Loss:   0.0228, Val Acc:   0.9735
Epoch 20/20 took 6.67s
  Train Loss: 0.0062, Train Acc: 0.9938
  Val Loss:   0.0245, Val Acc:   0.9731

Starting final test evaluation...
  Test Loss: 0.0255
  Test Acc:  0.9720


In [14]:
import random
import time
import datetime
import numpy as np
import tensorflow as tf
import tensorlayer as tl
from tensorlayer.layers import Dense, Dropout, Flatten, Input
from tensorlayer.models import Model

# ==============================================================================

# ==============================================================================
EXPERIMENT_CONFIGS = [
    {'LR': 0.001, 'DROPOUT': 0.3, 'EMBEDDING': 128, 'MARGIN': 1.0},
    {'LR': 0.0001, 'DROPOUT': 0.3, 'EMBEDDING': 128, 'MARGIN': 1.0},
    {'LR': 0.001, 'DROPOUT': 0.5, 'EMBEDDING': 128, 'MARGIN': 1.0},
    {'LR': 0.001, 'DROPOUT': 0.3, 'EMBEDDING': 64, 'MARGIN': 1.0},
    {'LR': 0.001, 'DROPOUT': 0.3, 'EMBEDDING': 128, 'MARGIN': 1.5},
]

EPOCHS = 20
BATCH_SIZE = 128
NUM_CLASSES = 10
INPUT_SHAPE = (None, 784)
PRINT_FREQ = 5

# ==============================================================================
# ==============================================================================

def contrastive_loss(label, feature1, feature2, margin):
    eucd = tf.sqrt(tf.reduce_sum(tf.square(feature1 - feature2), axis=1))
    return tf.reduce_mean(label * tf.square(eucd) + (1 - label) * tf.square(tf.maximum(margin - eucd, 0)))

def compute_accuracy(label, feature1, feature2, margin):
    eucd = tf.sqrt(tf.reduce_sum((feature1 - feature2)**2, axis=1))
    pred = tf.cast(eucd < (margin / 2.0), label.dtype)
    return tf.reduce_mean(tf.cast(tf.equal(pred, label), tf.float32))

# --- معماری شبکه ---
def create_base_network(input_shape, embedding_units, dropout_rate):
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(embedding_units, act=tf.nn.relu)(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(embedding_units, act=tf.nn.relu)(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(embedding_units, act=tf.nn.relu)(x)
    return Model(input, x)

def get_siamese_network(input_shape, embedding_units, dropout_rate):
    base_layer = create_base_network(input_shape, embedding_units, dropout_rate).as_layer()
    ni_1 = Input(input_shape)
    ni_2 = Input(input_shape)
    nn_1 = base_layer(ni_1)
    nn_2 = base_layer(ni_2)
    return Model(inputs=[ni_1, ni_2], outputs=[nn_1, nn_2])

def create_pairs(x, digit_indices):
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(NUM_CLASSES)]) - 1
    for d in range(NUM_CLASSES):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, NUM_CLASSES)
            dn = (d + inc) % NUM_CLASSES
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels).astype(np.float32)

# ==============================================================================
# ==============================================================================
def load_and_prep_data():
    tl.logging.info("Loading and preprocessing data... (This happens only once)")
    try:
        (X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
        X_train = X_train.reshape(-1, 784).astype(np.float32) / 255.0
        X_test = X_test.reshape(-1, 784).astype(np.float32) / 255.0
        y_train = y_train.astype(np.int64)
        y_test = y_test.astype(np.int64)

        X_val = X_train[50000:]
        y_val = y_train[50000:]
        X_train = X_train[:50000]
        y_train = y_train[:50000]
    except Exception as e:
        tl.logging.error(f"Failed to load data: {e}")
        return None

    digit_indices = [np.where(y_train == i)[0] for i in range(NUM_CLASSES)]
    tr_pairs, tr_y = create_pairs(X_train, digit_indices)
    digit_indices = [np.where(y_val == i)[0] for i in range(NUM_CLASSES)]
    val_pairs, val_y = create_pairs(X_val, digit_indices)
    digit_indices = [np.where(y_test == i)[0] for i in range(NUM_CLASSES)]
    te_pairs, te_y = create_pairs(X_test, digit_indices)

    return (tr_pairs, tr_y), (val_pairs, val_y), (te_pairs, te_y)

# ==============================================================================
# ==============================================================================
def run_experiment(params, prepared_data):
    LR = params['LR']
    DROPOUT = params['DROPOUT']
    EMBEDDING = params['EMBEDDING']
    MARGIN = params['MARGIN']

    (tr_pairs, tr_y), (val_pairs, val_y), (te_pairs, te_y) = prepared_data

    model = get_siamese_network(INPUT_SHAPE, EMBEDDING, DROPOUT)
    train_weights = model.trainable_weights
    optimizer = tf.optimizers.RMSprop(learning_rate=LR)

    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    run_name = f"LR={LR}_DR={DROPOUT}_E={EMBEDDING}_M={MARGIN}"
    log_dir = f'logs/siamese/{current_time}_{run_name}'
    train_summary_writer = tf.summary.create_file_writer(log_dir + '/train')
    val_summary_writer = tf.summary.create_file_writer(log_dir + '/validation')
    tl.logging.info(f"TensorBoard log for this run: {log_dir}")

    @tf.function
    def train_step(X_batch, y_batch):
        with tf.GradientTape() as tape:
            _out1, _out2 = model([X_batch[:, 0, :], X_batch[:, 1, :]])
            _loss = contrastive_loss(y_batch, _out1, _out2, margin=MARGIN)
        grad = tape.gradient(_loss, train_weights)
        optimizer.apply_gradients(zip(grad, train_weights))
        _acc = compute_accuracy(y_batch, _out1, _out2, margin=MARGIN)
        return _loss, _acc

    @tf.function
    def eval_step(X_batch, y_batch):
        _out1, _out2 = model([X_batch[:, 0, :], X_batch[:, 1, :]])
        _loss = contrastive_loss(y_batch, _out1, _out2, margin=MARGIN)
        _acc = compute_accuracy(y_batch, _out1, _out2, margin=MARGIN)
        return _loss, _acc

    best_val_acc = 0.0
    for epoch in range(EPOCHS):
        start_time = time.time()

        train_loss, train_acc, n_iter = 0, 0, 0
        model.train()
        for X_batch, y_batch in tl.iterate.minibatches(tr_pairs, tr_y, BATCH_SIZE, shuffle=True):
            _loss, _acc = train_step(X_batch, y_batch)
            train_loss += _loss
            train_acc += _acc
            n_iter += 1
        avg_train_loss = train_loss / n_iter
        avg_train_acc = train_acc / n_iter

        with train_summary_writer.as_default():
            tf.summary.scalar('loss', avg_train_loss, step=epoch)
            tf.summary.scalar('accuracy', avg_train_acc, step=epoch)

        val_loss, val_acc, n_iter_val = 0, 0, 0
        model.eval()
        for X_batch, y_batch in tl.iterate.minibatches(val_pairs, val_y, BATCH_SIZE, shuffle=False):
            _loss, _acc = eval_step(X_batch, y_batch)
            val_loss += _loss
            val_acc += _acc
            n_iter_val += 1
        avg_val_loss = val_loss / n_iter_val
        avg_val_acc = val_acc / n_iter_val

        if avg_val_acc > best_val_acc:
            best_val_acc = avg_val_acc

        with val_summary_writer.as_default():
            tf.summary.scalar('loss', avg_val_loss, step=epoch)
            tf.summary.scalar('accuracy', avg_val_acc, step=epoch)

        if epoch + 1 == 1 or (epoch + 1) % PRINT_FREQ == 0:
            print(f"  Epoch {epoch + 1}/{EPOCHS} - Train Acc: {avg_train_acc:.4f} - Val Acc: {avg_val_acc:.4f}")

    test_loss, test_acc, n_iter_test = 0, 0, 0
    model.eval()
    for X_batch, y_batch in tl.iterate.minibatches(te_pairs, te_y, BATCH_SIZE, shuffle=False):
        _loss, _acc = eval_step(X_batch, y_batch)
        test_loss += _loss
        test_acc += _acc
        n_iter_test += 1

    final_test_loss = test_loss / n_iter_test
    final_test_acc = test_acc / n_iter_test

    print(f"  Final Test Acc: {final_test_acc:.4f}, Best Val Acc: {best_val_acc:.4f}")

    return final_test_loss, final_test_acc, best_val_acc

# ==============================================================================
# ==============================================================================
def print_results_table(results):
    print("\n\n" + "="*80)
    print("🏁 FINAL EXPERIMENT RESULTS 🏁")
    print("="*80)

    header = "| # | LR     | Dropout | Embedding | Margin | Test Acc | Test Loss | Best Val Acc |"
    print(header)
    print("-" * len(header))

    sorted_results = sorted(results, key=lambda x: x['test_acc'], reverse=True)

    for i, res in enumerate(sorted_results):
        p = res['params']
        print(
            f"| {i+1:<1} "
            f"| {p['LR']:<6} "
            f"| {p['DROPOUT']:<7} "
            f"| {p['EMBEDDING']:<9} "
            f"| {p['MARGIN']:<6} "
            f"| {res['test_acc'] * 100:6.2f}%   "  # نمایش به صورت درصد
            f"| {res['test_loss']:9.4f} "
            f"| {res['val_acc'] * 100:10.2f}%   |"
        )
    print("-" * len(header))


# ==============================================================================
# ==============================================================================
if __name__ == "__main__":

    prepared_data = load_and_prep_data()
    if prepared_data is None:
        print("Failed to load data, exiting.")
        exit()

    final_results = []

    for i, params in enumerate(EXPERIMENT_CONFIGS):
        print("\n" + "="*50)
        print(f"🚀 STARTING EXPERIMENT {i+1}/{len(EXPERIMENT_CONFIGS)}")
        print(f"Parameters: {params}")
        print("="*50)

        test_loss, test_acc, val_acc = run_experiment(params, prepared_data)

        result_data = {
            'params': params,
            'test_loss': float(test_loss),
            'test_acc': float(test_acc),
            'val_acc': float(val_acc)
        }
        final_results.append(result_data)

    print_results_table(final_results)


🚀 STARTING EXPERIMENT 1/5
Parameters: {'LR': 0.001, 'DROPOUT': 0.3, 'EMBEDDING': 128, 'MARGIN': 1.0}
  Epoch 1/20 - Train Acc: 0.5147 - Val Acc: 0.6571
  Epoch 5/20 - Train Acc: 0.8178 - Val Acc: 0.8555
  Epoch 10/20 - Train Acc: 0.8388 - Val Acc: 0.8846
  Epoch 15/20 - Train Acc: 0.6510 - Val Acc: 0.5000
  Epoch 20/20 - Train Acc: 0.4999 - Val Acc: 0.5000
  Final Test Acc: 0.5000, Best Val Acc: 0.8894

🚀 STARTING EXPERIMENT 2/5
Parameters: {'LR': 0.0001, 'DROPOUT': 0.3, 'EMBEDDING': 128, 'MARGIN': 1.0}
  Epoch 1/20 - Train Acc: 0.5021 - Val Acc: 0.5000
  Epoch 5/20 - Train Acc: 0.5109 - Val Acc: 0.5000
  Epoch 10/20 - Train Acc: 0.7324 - Val Acc: 0.8046
  Epoch 15/20 - Train Acc: 0.8138 - Val Acc: 0.8488
  Epoch 20/20 - Train Acc: 0.8365 - Val Acc: 0.8594
  Final Test Acc: 0.8576, Best Val Acc: 0.8594

🚀 STARTING EXPERIMENT 3/5
Parameters: {'LR': 0.001, 'DROPOUT': 0.5, 'EMBEDDING': 128, 'MARGIN': 1.0}
  Epoch 1/20 - Train Acc: 0.7476 - Val Acc: 0.8937
  Epoch 5/20 - Train Acc: 0.9236

<div dir="rtl">

##  تحلیل کوتاه نتایج آزمایش

* **پیکربندی برنده (رتبه ۱):**
    * **پارامترها:** `LR=0.001`, `Dropout=0.5`, `Embedding=128`
    * **نتیجه:** **۹۶.۲۳٪** دقت تست.
    * **تحلیل:** این مدل بهترین تعادل را داشت. `Dropout=0.5` (تنظیم‌کننده قوی) توانست نرخ یادگیری تهاجمی `LR=0.001` را مهار و پایدار کند و مانع از انفجار گرادیان شود.

* **مدل‌های شکست‌خورده (رتبه‌های ۳، ۴، ۵):**
    * **پارامترها:** همگی `LR=0.001` و `Dropout=0.3` (یا بدتر).
    * **نتیجه:** **۵۰.۰۰٪** دقت (معادل شانس) با `Test Loss = nan`.
    * **تحلیل:** این مدل‌ها دچار **انفجار گرادیان (Exploding Gradient)** شدند. ترکیب نرخ یادگیری بالا (`0.001`) با Dropout ناکافی (`0.3`) سمی بود و باعث فروپاشی کامل آموزش شد.

* **مدل کند (رتبه ۲):**
    * **پارامترها:** `LR=0.0001`, `Dropout=0.3`
    * **نتیجه:** **۸۵.۷۶٪** دقت تست.
    * **تحلیل:** این مدل پایدار بود اما به شدت **کم‌آموزش‌دیده (Under-trained)** است. نرخ یادگیری بسیار پایین آن باعث شد در ۲۰ Epoch به پتانسیل کامل خود نرسد.

</div>